<a href="https://colab.research.google.com/github/EstebanCabreraArbizu/TP-TF_Topicos_CC/blob/main/topicos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install python-constraint

# Descargar los datos de kaggle a google colab.
* Advrtencia: es necesario tener tu api (.json) de kaggle dentro de content en google colab

In [29]:
!pip install -q kaggle
import os
if not os.path.exists('/root/.kaggle'):
    !mkdir /root/.kaggle
if not os.path.isfile('/root/.kaggle/kaggle.json'):
    !cp kaggle.json /root/.kaggle/
    !chmod 600 /root/.kaggle/kaggle.json
if not os.path.isfile('/content/bus-transit-data.zip'):
  !kaggle datasets download -d lyxbash/bus-transit-data
  !unzip "/content/bus-transit-data.zip" -d "/content/"

In [28]:
import pandas as pd
import time

# Medimos el tiempo para cargar los datos
start_time = time.time()

# Cargamos los datos con tipos optimizados
routes_df = pd.read_csv('routes.txt', dtype={'route_id': 'int32'})
shapes_df = pd.read_csv('shapes.txt', dtype={'shape_id': 'str', 'shape_pt_lat': 'float32', 'shape_pt_lon': 'float32', 'shape_pt_sequence': 'int32'})
stop_times_df = pd.read_csv('stop_times.txt', dtype={'trip_id': 'str', 'stop_id': 'int32', 'stop_sequence': 'int32'})
stops_df = pd.read_csv('stops.txt', dtype={'stop_id': 'int32', 'stop_lat': 'float32', 'stop_lon': 'float32'})
trips_df = pd.read_csv('trips.txt', dtype={'trip_id': 'str', 'route_id': 'int32', 'shape_id': 'str'})

# Mide el tiempo de carga de los datos
print(f"Tiempo para cargar los datos: {time.time() - start_time} segundos")

Tiempo para cargar los datos: 0.39658498764038086 segundos


# Implementación de CP en los tiempos de parada y rutas de los buses

In [ ]:
!pip install ortools

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 28.1/28.1 MB 61.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.7/133.7 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.8/302.8 kB 23.0 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.20.3
    Uninstalling protobuf-3.20.3:
      Successfully uninstalled protobuf-3.20.3
  Attempting uninstall: absl-py
    Found existing installation: absl-py 1.4.0
    Uninstalling absl-py-1.4.0:
      Successfully uninstalled absl-py-1.4.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires protobuf<5,>=3.20, but you have protobuf 5.26.1 which is incompatible.
google-ai-generativelanguage 0.6.6 requires protobuf!=3.20.0,!=3.20.1,!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<5.0.0dev,>=3.19.5, but you have protobuf 5.2

## Primera versión

In [ ]:
from ortools.sat.python import cp_model
import pandas as pd
from datetime import timedelta
import time

# Medir tiempo de inicio
start_time = time.time()

# Cargar los datasets
stop_times_df = pd.read_csv('stop_times.txt')
trips_df = pd.read_csv('trips.txt')

# Función para ajustar tiempos inválidos que exceden 24 horas
def adjust_time_format(time_str):
    # Dividir la cadena de tiempo en horas, minutos y segundos
    time_parts = time_str.split(':')
    hours = int(time_parts[0])
    minutes = int(time_parts[1])
    seconds = int(time_parts[2])

    # Si las horas son mayores o iguales a 24, ajustarlas
    if hours >= 24:
        hours -= 24  # Ajustar a formato de 24 horas
        return f'{hours:02}:{minutes:02}:{seconds:02}'
    else:
        return time_str

# Aplicar la función de ajuste al campo de tiempos de llegada y salida
stop_times_df['arrival_time'] = stop_times_df['arrival_time'].apply(adjust_time_format)
stop_times_df['departure_time'] = stop_times_df['departure_time'].apply(adjust_time_format)

# Convertir los tiempos a formato datetime
stop_times_df['arrival_time'] = pd.to_datetime(stop_times_df['arrival_time'], format='%H:%M:%S')
stop_times_df['departure_time'] = pd.to_datetime(stop_times_df['departure_time'], format='%H:%M:%S')

# Crear una instancia del modelo CSP de OR-Tools
model = cp_model.CpModel()

# Variables
trip_ids = trips_df['trip_id'].unique()
stop_ids = stop_times_df['stop_id'].unique()

# Crear las variables de decisión
trip_variables = {}
for trip_id in trip_ids:
    for stop_id in stop_times_df[stop_times_df['trip_id'] == trip_id]['stop_id'].unique():
        # Definimos una variable binaria que indica si un trip_id pasa por un stop_id
        trip_variables[(trip_id, stop_id)] = model.NewBoolVar(f'trip_{trip_id}_stop_{stop_id}')

# Restricción 1: Secuencia de paradas correcta
for trip_id in trip_ids:
    trip_stop_times = stop_times_df[stop_times_df['trip_id'] == trip_id].sort_values(by='stop_sequence')
    stop_sequence = trip_stop_times['stop_id'].tolist()

    for i in range(len(stop_sequence) - 1):
        stop1 = stop_sequence[i]
        stop2 = stop_sequence[i + 1]

        # Verificamos que ambas paradas existan en trip_variables antes de aplicar la restricción
        if (trip_id, stop1) in trip_variables and (trip_id, stop2) in trip_variables:
            # La parada siguiente debe ser mayor en la secuencia
            model.Add(trip_variables[(trip_id, stop1)] <= trip_variables[(trip_id, stop2)])

# Restricción 2: Respetar los tiempos de llegada y salida
for trip_id in trip_ids:
    trip_times = stop_times_df[stop_times_df['trip_id'] == trip_id]
    arrival_times = trip_times['arrival_time']
    departure_times = trip_times['departure_time']

    # Aseguramos que la hora de salida sea mayor o igual que la de llegada en cada parada
    for i in range(len(arrival_times)):
        if i < len(departure_times) and i < len(arrival_times):
            model.Add(arrival_times.iloc[i] <= departure_times.iloc[i])

# Crear un solver para resolver el modelo
solver = cp_model.CpSolver()
status = solver.Solve(model)

# Verificar si se encontró una solución
if status == cp_model.FEASIBLE:
    print("Se encontró una solución:")
    for trip_id in trip_ids:
        for stop_id in stop_ids:
            if (trip_id, stop_id) in trip_variables and solver.Value(trip_variables[(trip_id, stop_id)]) == 1:
                print(f"El trip_id {trip_id} pasa por la parada {stop_id}")
else:
    print("No se encontró una solución factible.")

# Mostrar tiempo de ejecución
print(f"Tiempo total de ejecución: {time.time() - start_time} segundos")



No se encontró una solución factible.
Tiempo total de ejecución: 27.332876920700073 segundos


## Segunda versión
Que tiene resultados

In [ ]:
from ortools.sat.python import cp_model
import pandas as pd
from datetime import timedelta

# Cargar los datasets
routes_df = pd.read_csv('routes.txt')
stop_times_df = pd.read_csv('stop_times.txt')
trips_df = pd.read_csv('trips.txt')

# Función para ajustar los tiempos que exceden las 24 horas
def adjust_time_format(time_str):
    time_parts = time_str.split(':')
    hours = int(time_parts[0])
    minutes = int(time_parts[1])
    seconds = int(time_parts[2])

    # Ajustar horas mayores a 23
    if hours >= 24:
        hours -= 24  # Convertir el tiempo a formato de 24 horas
    return f'{hours:02}:{minutes:02}:{seconds:02}'

# Aplicar la función para ajustar los tiempos en el dataset
stop_times_df['arrival_time'] = stop_times_df['arrival_time'].apply(adjust_time_format)
stop_times_df['departure_time'] = stop_times_df['departure_time'].apply(adjust_time_format)

# Convertir a formato datetime después del ajuste
stop_times_df['arrival_time'] = pd.to_datetime(stop_times_df['arrival_time'], format='%H:%M:%S')
stop_times_df['departure_time'] = pd.to_datetime(stop_times_df['departure_time'], format='%H:%M:%S')

# Extraer los tiempos de viaje de las rutas a partir de stop_times.txt
viajes_tiempos = {}
for trip_id in trips_df['trip_id'].unique():
    # Filtrar por trip_id para calcular el tiempo de viaje total
    trip_times = stop_times_df[stop_times_df['trip_id'] == trip_id]
    if len(trip_times) > 1:
        tiempo_inicio = trip_times['arrival_time'].iloc[0]
        tiempo_final = trip_times['departure_time'].iloc[-1]
        duracion_viaje = (tiempo_final - tiempo_inicio).seconds // 60  # Duración en minutos
        viajes_tiempos[trip_id] = duracion_viaje

# Definir las rutas desde el dataset
rutas = routes_df['route_id'].unique()

# Definir el número de vehículos y sus capacidades
num_vehiculos = 8  # Número de vehículos disponibles
capacidad_vehiculos = [50, 80, 70, 70, 60,50,80,70]  # Capacidad máxima de cada vehículo

# Definir tiempos máximos de viaje (por ejemplo, 150 minutos)
max_tiempo_viaje = 200

# Crear una instancia del modelo CSP de OR-Tools
model = cp_model.CpModel()

# Crear variables de asignación de vehículos a rutas
vehiculo_ruta = {}
for vehiculo in range(num_vehiculos):
    for ruta in rutas:
        vehiculo_ruta[(vehiculo, ruta)] = model.NewBoolVar(f"vehiculo_{vehiculo}_ruta_{ruta}")

# Crear variables de horarios
horarios = range(6, 22)  # Horarios de operación de los vehículos (de 6 AM a 10 PM)
vehiculo_horario = {}
for vehiculo in range(num_vehiculos):
    for horario in horarios:
        vehiculo_horario[(vehiculo, horario)] = model.NewBoolVar(f"vehiculo_{vehiculo}_horario_{horario}")

# Restricción 1: Cada vehículo cubre solo una ruta


# Restricción 2: Cada ruta debe ser cubierta por al menos un vehículo
for ruta in rutas:
    model.Add(sum(vehiculo_ruta[(vehiculo, ruta)] for vehiculo in range(num_vehiculos)) >= 1)

# Restricción 3: Los vehículos deben operar durante las horas pico
horas_pico = [7, 8, 9]
for vehiculo in range(num_vehiculos):
    model.Add(sum(vehiculo_horario[(vehiculo, hora)] for hora in horas_pico) >= 1)

# Restricción 4: Los vehículos no deben exceder el tiempo máximo de viaje permitido
for trip_id, duracion in viajes_tiempos.items():
    if duracion > max_tiempo_viaje:
        print(f"El viaje {trip_id} excede el tiempo máximo permitido de {max_tiempo_viaje} minutos.")

# Restricción 5: Capacidad de vehículos
for vehiculo in range(num_vehiculos):
    capacidad_max = capacidad_vehiculos[vehiculo]
    # Ejemplo simplificado para la restricción de capacidad
    model.Add(capacidad_max <= 80)  # Limitar la capacidad máxima a 60 pasajeros

# Función de optimización: minimizar el tiempo total de viaje
total_tiempo_viaje = sum(vehiculo_ruta[(vehiculo, ruta)] * max_tiempo_viaje for vehiculo in range(num_vehiculos) for ruta in rutas)
model.Minimize(total_tiempo_viaje)

# Crear el solver y resolver el modelo
solver = cp_model.CpSolver()
status = solver.Solve(model)

# Verificar si se encontró una solución
if status == cp_model.OPTIMAL or status == cp_model.FEASIBLE:
    print("Solución óptima encontrada:")
    for vehiculo in range(num_vehiculos):
        for ruta in rutas:
            if solver.Value(vehiculo_ruta[(vehiculo, ruta)]) == 1:
                print(f"El vehículo {vehiculo} cubre la ruta {ruta}.")
else:
    print("No se encontró una solución factible.")

Solución óptima encontrada:
El vehículo 7 cubre la ruta 33.
El vehículo 7 cubre la ruta 31.
El vehículo 7 cubre la ruta 29.
El vehículo 7 cubre la ruta 12.
El vehículo 7 cubre la ruta 11.
El vehículo 7 cubre la ruta 18.
El vehículo 7 cubre la ruta 9.
El vehículo 7 cubre la ruta 1.
El vehículo 7 cubre la ruta 22.
El vehículo 7 cubre la ruta 8.
El vehículo 7 cubre la ruta 0.
El vehículo 7 cubre la ruta 17.
El vehículo 7 cubre la ruta 28.
El vehículo 7 cubre la ruta 19.
El vehículo 7 cubre la ruta 4.
El vehículo 7 cubre la ruta 5.
El vehículo 7 cubre la ruta 30.
El vehículo 7 cubre la ruta 6.
El vehículo 7 cubre la ruta 3.
El vehículo 7 cubre la ruta 16.
El vehículo 7 cubre la ruta 7.
El vehículo 7 cubre la ruta 10.
El vehículo 7 cubre la ruta 35.
El vehículo 7 cubre la ruta 27.
El vehículo 7 cubre la ruta 13.
El vehículo 7 cubre la ruta 15.
El vehículo 7 cubre la ruta 34.
El vehículo 7 cubre la ruta 14.
El vehículo 7 cubre la ruta 32.
El vehículo 7 cubre la ruta 26.
El vehículo 7 cubre l